<a href="https://colab.research.google.com/github/Samar-Agarwal/Detecting-Depression-through-Tweets/blob/main/wids_nlp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data =pd.read_csv("drive/MyDrive/WIDS_NLP_Project/dataset2.csv", on_bad_lines = 'skip')
# Keeping only the neccessary columns
data = data[['Sentiment', 'SentimentText']]

In [ ]:
data['SentimentText'] = data['SentimentText'].apply(lambda x: x.lower()) # lowering all alphabets
data['SentimentText'] = data['SentimentText'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))  #using re module for stooping

print(data[ data['Sentiment'] == 1].size)
print(data[ data['Sentiment'] == 0].size)

1580354
1576870


In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)

In [ ]:
from keras.models import Sequential
from keras.layers import SpatialDropout1D
embed_dim = 50
lstm_out = 80
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 50)            100000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 40, 50)           0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 80)                41920     
                                                                 
 dense (Dense)               (None, 2)                 162       
                                                                 
Total params: 142,082
Trainable params: 142,082
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
data.head()
from sklearn.model_selection import train_test_split

In [ ]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1105028, 40) (1105028, 2)
(473584, 40) (473584, 2)


In [ ]:
batch_size = 32
epochs = 1
model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, verbose=2)

34533/34533 - 2349s - loss: 0.4276 - accuracy: 0.8010 - 2349s/epoch - 68ms/step


In [ ]:
model.metrics_names


['loss', 'accuracy']

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

14800/14800 - 156s - loss: 0.4203 - accuracy: 0.8048 - 156s/epoch - 11ms/step
score: 0.42
acc: 0.80
